In [305]:
from IPython.display import Markdown as md
import pandas as pd
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats.mstats import gmean 
from datetime import datetime as dt

%run ../COMMON/common.ipynb # загрузка общих функций и констант, все, что оттуда, должно иметь префикс common.

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strYearDBPath)) # connection к рабочей базе данных
conWorkEx = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExYearDBPath)) # connection к к рабочей базе данных экзогенных переменных
conWorkExH=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExParamDBPath)) # connection к рабочей базе данных экзогенных параметров
conWorkSvod=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strSvodDBPath)) # connection к рабочей базе данных СВОД


iFirstFactYear=2005
iLastFactYear=2019

iFirstForecastYear=iLastFactYear+1
iLastForecastYear=2030

In [306]:
class finance:
    pdfWork=None
    
    lstYearCodes=['CPIAv','KeyRate','developers_ss_s','guarantee_coverage','loans_secured',
                  'rw_loans_secured','rw_loans_not_secured','h1','ROE','dom_market_share',
                  'contributions_rate_ASV','contributions_rate_FOR','Vvody','VvodyMKD_private',
                  'ProjectsPrivate_x','ProjectsPrivate_escrow_s','ConstrSales_s','seb1m_full_x',
                  'price1mperv_x','loans_and_ref_vol','loans_vol_MKD_perv'] # список экзогенных и фактических данных
 
    def __init__(self):
        assert False, 'you can\'t create variables of repay_e class!'
    
    def MakeWorkFrame():
        ''' Загрузка исходных данных
            Исходные данные:
                Фактические значения - из базы данных year.sqlite3;
                Экзогенные - из базы данных exog_year.sqlite3
                Задаваемые вручную - из базы данных exog_param.sqlite3
                Результаты других моделей - из базы данных svod.sqlite3. 
        '''
        
        pdfAct=(pd.read_sql(common.make_SELECT_YEAR_string(finance.lstYearCodes), con=conWork)
        .pipe(common.make_frame)
        .pipe(common.scale, list_fields=['KeyRate'], multiplier=1e2)
        .pipe(common.scale, list_fields=['loans_vol_MKD_perv'], multiplier=1e-3)) 
        
        pdfExog=(pd.read_sql(common.make_SELECT_YEAR_string(finance.lstYearCodes), con=conWorkEx)
        .pipe(common.make_frame))
        
        PdfExogHandle=(pd.read_sql(common.make_SELECT_YEAR_string(finance.lstYearCodes), con=conWorkExH)
        .pipe(common.make_frame))
        
        pdfSvod=(pd.read_sql(common.make_SELECT_YEAR_string(finance.lstYearCodes), con=conWorkSvod)
        .pipe(common.make_frame)
        .pipe(common.scale, list_fields=['loans_and_ref_vol','loans_vol_MKD_perv'], multiplier=1e3))
        
        finance.pdfWork=pdfAct.combine_first(pdfExog).combine_first(PdfExogHandle).combine_first(pdfSvod)

        finance.pdfWork['ProjectsPrivate_escrow_x']=finance.pdfWork['ProjectsPrivate_x']*finance.pdfWork['ProjectsPrivate_escrow_s']/100
        
        return finance.pdfWork

In [307]:
finance.pdfWork = finance.MakeWorkFrame()
finance.pdfWork

,CPIAv,ConstrSales_s,KeyRate,ProjectsPrivate_escrow_s,ProjectsPrivate_x,ROE,Vvody,VvodyMKD_private,contributions_rate_ASV,contributions_rate_FOR,...,guarantee_coverage,h1,loans_and_ref_vol,loans_secured,loans_vol_MKD_perv,price1mperv_x,rw_loans_not_secured,rw_loans_secured,seb1m_full_x,ProjectsPrivate_escrow_x
date,,,,,,,,,,,,,,,,,,,,,
2004,1.108812,NaN,13.520000,NaN,27.254825,NaN,NaN,22.167600,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,1.127055,NaN,12.980000,NaN,30.726575,0.15,43.559500,23.249900,0.006,0.0475,...,0.8,0.100,NaN,NaN,NaN,23.771597,1.0,0.2,NaN,NaN
2006,1.096830,NaN,8.894216,NaN,31.297480,0.15,50.552100,27.301900,0.006,0.0475,...,0.8,0.100,NaN,NaN,NaN,31.473560,1.0,0.2,NaN,NaN
2007,1.089881,NaN,7.512557,NaN,29.248635,0.15,61.221300,31.165600,0.006,0.0475,...,0.8,0.100,NaN,NaN,NaN,43.883193,1.0,0.2,NaN,NaN
2008,1.141048,NaN,8.129107,NaN,29.003730,0.15,64.058400,33.273200,0.006,0.0475,...,0.8,0.100,655.808000,NaN,121.000000,51.332988,1.0,0.2,NaN,NaN
2009,1.116977,NaN,11.310000,NaN,31.282002,0.15,59.891560,27.477920,0.006,0.0475,...,0.8,0.100,152.501000,NaN,16.000000,48.865360,1.0,0.2,NaN,NaN
2010,1.068508,NaN,8.030000,NaN,34.126605,0.15,58.430700,28.765500,0.006,0.0475,...,0.8,0.100,380.061000,NaN,39.000000,48.122768,1.0,0.2,NaN,NaN
2011,1.084634,NaN,8.120000,NaN,38.266952,0.15,62.264600,31.006000,0.006,0.0475,...,0.8,0.100,716.944000,NaN,108.000000,42.581430,1.0,0.2,NaN,NaN
2012,1.050667,0.354046,8.070000,NaN,43.570800,0.15,65.741510,34.350510,0.006,0.0475,...,0.8,0.100,1031.992000,NaN,206.000000,46.740097,1.0,0.2,NaN,NaN


In [325]:
#     portf 2019 должен приходить из базы
finance.pdfWork.loc[2019,'ddy_portf']  = 3157.7

finance.pdfWork.loc[2019,'escrow_sales']=142.3
finance.pdfWork.loc[2019:2020,'escrow_disclosure']=[5,10]
finance.pdfWork.loc[2018:2019,'escrow_funds']=[0,142.3]
finance.pdfWork.loc[2018,'escrow_projects_budget']=0

finance.pdfWork['perv_in_ipot_s']=finance.pdfWork['loans_vol_MKD_perv']/finance.pdfWork['loans_and_ref_vol']

finance.pdfWork.loc[2020:,'escrow_sales'] = (finance.pdfWork['ConstrSales_s'] * 
                                             (0.25*finance.pdfWork['price1mperv_x'].shift(2)*finance.pdfWork['ProjectsPrivate_escrow_x'].shift(2) + 
                                              0.5 *finance.pdfWork['price1mperv_x'].shift(1)*finance.pdfWork['ProjectsPrivate_escrow_x'].shift(1) + 
                                              0.25*finance.pdfWork['price1mperv_x']*finance.pdfWork['ProjectsPrivate_escrow_x']))

finance.pdfWork.loc[2021:,'escrow_disclosure'] = finance.pdfWork[['ConstrSales_s', 
                                                                  'price1mperv_x', 
                                                                  'ProjectsPrivate_escrow_x']].shift(2).prod(axis=1)


for i in range (iFirstForecastYear-2,iLastForecastYear+1):
    finance.pdfWork.loc[i,'ddy_vvody'] =(finance.pdfWork.loc[i,'ProjectsPrivate_x'] -
                                         finance.pdfWork.loc[i,'ProjectsPrivate_escrow_x'])
    
    if i==2021:
        finance.pdfWork.loc[i,'ddy_sales'] = 0 #уточнить не ошибка ли это
        finance.pdfWork.loc[i,'ddy_disps']  = (finance.pdfWork.loc[i,'ddy_sales'] + 
                                               finance.pdfWork.loc[i-1,'ddy_portf'])
    
    else:
        finance.pdfWork.loc[i,'ddy_sales'] = (finance.pdfWork.loc[i,'ConstrSales_s'] * 
                                              (0.25*finance.pdfWork.loc[i-2,['price1mperv_x','ddy_vvody']].fillna(value=0).prod() + 
                                               0.5 *finance.pdfWork.loc[i-1,['price1mperv_x','ddy_vvody']].fillna(value=0).prod() + 
                                               0.25*finance.pdfWork.loc[i,  ['price1mperv_x','ddy_vvody']].fillna(value=0).prod()))
        if i>2019:
            finance.pdfWork.loc[i,'ddy_disps'] = finance.pdfWork.loc[i-2,['ConstrSales_s', 
                                                                          'price1mperv_x', 
                                                                          'ddy_vvody']].prod()
    if i>2019:
        finance.pdfWork.loc[i,'ddy_portf']  = (finance.pdfWork.loc[i-1,'ddy_portf'] + 
                                               finance.pdfWork.loc[i  ,'ddy_sales'] - 
                                               finance.pdfWork.loc[i  ,'ddy_disps'])

        finance.pdfWork.loc[i,'escrow_funds']  = (finance.pdfWork.loc[i-1,'escrow_funds'] + 
                                                  finance.pdfWork.loc[i  ,'escrow_sales'] - 
                                                  finance.pdfWork.loc[i  ,'escrow_disclosure'])

# for i in range (iFirstForecastYear,iLastForecastYear+1):
#     finance.pdfWork.loc[i,'escrow_sales'] = (finance.pdfWork.loc[i,'ConstrSales_s'] * 
#                                              (0.25*finance.pdfWork.loc[i-2,['price1mperv_x','ProjectsPrivate_escrow_x']].fillna(value=0).prod() + 
#                                               0.5 *finance.pdfWork.loc[i-1,['price1mperv_x','ProjectsPrivate_escrow_x']].fillna(value=0).prod() + 
#                                               0.25*finance.pdfWork.loc[i,  ['price1mperv_x','ProjectsPrivate_escrow_x']].fillna(value=0).prod()))
#     if i>2020:
#         finance.pdfWork.loc[i,'escrow_disclosure'] = finance.pdfWork.loc[i-2,['ConstrSales_s', 
#                                                                               'price1mperv_x', 
#                                                                               'ProjectsPrivate_escrow_x']].prod()
 
#     finance.pdfWork.loc[i,'escrow_funds']  = (finance.pdfWork.loc[i-1,'escrow_funds'] + 
#                                               finance.pdfWork.loc[i  ,'escrow_sales'] - 
#                                               finance.pdfWork.loc[i  ,'escrow_disclosure'])

# остаются нули после 2030 г.
finance.pdfWork.loc[2019:,'tot_sales']=finance.pdfWork[['ddy_sales','escrow_sales']].sum(axis=1)

finance.pdfWork['contributions_ASV']=finance.pdfWork['escrow_funds'].rolling(2).mean()*finance.pdfWork['contributions_rate_ASV']
finance.pdfWork['contributions_FOR']=finance.pdfWork['escrow_funds'].rolling(2).mean()*finance.pdfWork['contributions_rate_FOR']*finance.pdfWork['KeyRate']/100

finance.pdfWork.loc[2019:,'escrow_projects_budget'] = finance.pdfWork['ProjectsPrivate_escrow_x']*finance.pdfWork['seb1m_full_x'].shift(-3).rolling(2).mean()
finance.pdfWork.loc[2019:,'projects_budget']        = finance.pdfWork['escrow_projects_budget'].rolling(2).sum()
finance.pdfWork.loc[2019:,'budget_funding']         =(finance.pdfWork['escrow_projects_budget'].shift(1)*0.75 + 
                                                      finance.pdfWork['escrow_projects_budget']*0.25)

finance.pdfWork.loc[2019:,'loans_credit_limit'] = finance.pdfWork['projects_budget']*(1-finance.pdfWork['developers_ss_s'])

finance.pdfWork.loc[2019,'guarantee_limit'] = ((finance.pdfWork.loc[2019,'ProjectsPrivate_escrow_x'] *
                                                finance.pdfWork.loc[2019,'loans_secured'] *
                                                finance.pdfWork.loc[2021,'seb1m_full_x'])*
                                                      finance.pdfWork.loc[2020,'guarantee_coverage']*
                                                      (1-finance.pdfWork.loc[2020,'developers_ss_s']))

finance.pdfWork.loc[2020:,'guarantee_limit'] = ((finance.pdfWork['ProjectsPrivate_escrow_x'].shift(1) *
                                                finance.pdfWork['loans_secured'].shift(1) *
                                                finance.pdfWork['seb1m_full_x'].shift(-1) +
                                                finance.pdfWork['ProjectsPrivate_escrow_x'] *
                                                finance.pdfWork['loans_secured'] *
                                                finance.pdfWork['seb1m_full_x'].shift(-2))*
                                                      finance.pdfWork['guarantee_coverage']*
                                                      (1-finance.pdfWork['developers_ss_s']))


finance.pdfWork.loc[2019:,'approved_app_limit']=(finance.pdfWork['loans_credit_limit']*
                                                 finance.pdfWork['dom_market_share'])

finance.pdfWork.loc[2019,'guarantees_issued']      =(finance.pdfWork.loc[2019,'ProjectsPrivate_escrow_x'] *
                                                      finance.pdfWork.loc[2019,'loans_secured'] *
                                                      finance.pdfWork.loc[2021,'seb1m_full_x']*
                                                      0.125*
                                                      finance.pdfWork.loc[2020,'guarantee_coverage']*
                                                      (1-finance.pdfWork.loc[2020,'developers_ss_s']))

finance.pdfWork.loc[2020:,'guarantees_issued'] = ((finance.pdfWork['ProjectsPrivate_escrow_x'].shift(1) *
                                                finance.pdfWork['loans_secured'].shift(1) *
                                                finance.pdfWork['seb1m_full_x'].shift(-1)*
                                                   0.75 +
                                                finance.pdfWork['ProjectsPrivate_escrow_x'] *
                                                finance.pdfWork['loans_secured'] *
                                                finance.pdfWork['seb1m_full_x'].shift(-2)*
                                                  0.25)*
                                                      finance.pdfWork['guarantee_coverage']*
                                                      (1-finance.pdfWork['developers_ss_s']))

finance.pdfWork.loc[2019:,'developers_ss'] = finance.pdfWork['projects_budget']*finance.pdfWork['developers_ss_s']

finance.pdfWork.loc[2019:,'loan_indebtedness'] = finance.pdfWork['budget_funding'] - finance.pdfWork['developers_ss']

finance.pdfWork.loc[2019:,'banks_capital_req'] = (finance.pdfWork['guarantees_issued']*
                                                  finance.pdfWork['h1']*
                                                  finance.pdfWork['rw_loans_secured']+ 
                                                      (finance.pdfWork['loan_indebtedness'] - 
                                                       finance.pdfWork['guarantees_issued'])*
                                                      finance.pdfWork['h1']*
                                                      finance.pdfWork['rw_loans_not_secured'])

finance.pdfWork.loc[2019:,'developers_add_funds'] = finance.pdfWork['developers_ss_s'] * finance.pdfWork['ddy_portf']

finance.pdfWork.loc[2019:,'developers_tot_funds'] = finance.pdfWork['developers_add_funds'] + finance.pdfWork['developers_ss']

In [322]:
finance.pdfWork[['escrow_projects_budget','projects_budget','budget_funding','loans_credit_limit','guarantee_limit']]

,escrow_projects_budget,projects_budget,budget_funding,loans_credit_limit,guarantee_limit
date,,,,,
2004,NaN,NaN,NaN,NaN,NaN
2005,NaN,NaN,NaN,NaN,NaN
2006,NaN,NaN,NaN,NaN,NaN
2007,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,NaN,NaN,NaN
2009,NaN,NaN,NaN,NaN,NaN
2010,NaN,NaN,NaN,NaN,NaN
2011,NaN,NaN,NaN,NaN,NaN
2012,NaN,NaN,NaN,NaN,NaN


In [125]:
from scipy import signal
signal.get_window('triang', 3, fftbins=False)/2

array([0.25, 0.5 , 0.25])